In [ ]:
from download import parse_multiple_urls, download_youtube_content

# Your YouTube video URL
url_string = "https://www.youtube.com/watch?v=oHry5RRI4KU"

# Parse and validate the URL
urls = parse_multiple_urls(url_string)

# Download audio only (MP3)
x = download_youtube_content(
    urls,
    output_path="downloads",  # or any directory you want
    max_workers=1,            # single URL, so 1 worker is fine
    audio_only=True           # <— this makes it download MP3 audio only
)
x


In [ ]:
x[0]['info']['automatic_captions']['hi-orig']

In [ ]:
import re

def extract_word_timestamps(vtt_path):
    """
    Extract individual word timestamps from a VTT file that contains
    inline <timestamp><c>word</c> patterns.

    Returns a list of [timestamp, word] pairs.
    """
    results = []

    # pattern for <00:00:00.240><c>WORD</c>
    word_pattern = re.compile(r"<(\d\d:\d\d:\d\d\.\d+)><c>([^<]+)</c>")

    with open(vtt_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            # find all matches in line
            for match in word_pattern.findall(line):
                ts, word = match
                results.append([ts, word.strip()])

    return results

# Example usage:
word_list = extract_word_timestamps("downloads\Ind-Pak War, Indian Army Operations, Weapons & Kargil - Capt Yashika Tyagi ｜ FO352 Raj Shamani.mp3.hi.vtt")
word_list


In [2]:
import soundfile as sf

# Load the original MP3 file (first 300 seconds)
input_path = 'downloads/Ind-Pak War, Indian Army Operations, Weapons & Kargil - Capt Yashika Tyagi ｜ FO352 Raj Shamani.mp3.mp3'
output_path = 'audio.wav'

# Read only the first 300 seconds (5 minutes)
with sf.SoundFile(input_path) as f:
    samplerate = f.samplerate
    frames_to_read = int(30 * samplerate)
    audio = f.read(frames=frames_to_read)

# Save to WAV
sf.write(output_path, audio, samplerate)

In [1]:
from speechbrain.inference.separation import SepformerSeparation as separator
from speechbrain.utils.fetching import LocalStrategy
import torchaudio
import os
os.add_dll_directory("C:\\ffmpeg\\bin\\")

model = separator.from_hparams(
    source="speechbrain/sepformer-wsj02mix",
    savedir='pretrained_models/sepformer-wsj02mix',
    local_strategy=LocalStrategy.COPY_SKIP_CACHE,
    run_opts={"device":"cuda"},
)

# for custom file, change path
est_sources = model.separate_file(path='audio.wav')

c:\Users\chait\.conda\envs\venv\Lib\site-packages\huggingface_hub\file_download.py:937: FutureWarning: The `force_filename` parameter is deprecated as a new caching system, which keeps the filenames as they are on the Hub, is now in place.
  warnings.warn(
c:\Users\chait\.conda\envs\venv\Lib\site-packages\huggingface_hub\file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Resampling the audio from 48000 Hz to 8000 Hz


In [2]:
torchaudio.save("source1hat.wav", est_sources[:, :, 0].detach().cpu(), 8000)
torchaudio.save("source2hat.wav", est_sources[:, :, 1].detach().cpu(), 8000)